In [1]:
import pandas as pd
import re
import requests
from nltk.util import bigrams

import time
from datetime import datetime 

start_time = datetime.now() 

In [2]:
bef = pd.read_csv('before_api.csv')
bef

,sentence,no_change,summarizing,bigrams,root,NOUN,VERB,ADV,AUX,ADJ,...,sym_ampersand,sym_percend,sym_star,sym_tilda,sym_math,sym_logic,sym_brackets,sym_num,sym_quotes,sym_slashes
0,והנפיחות בעיניים?,1,0,1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"ככלל, כדאי להיות קשובים לגוף וכשאשר מופיעים סי...",0,1,25,1,12,3,7,0,1,...,0,0,0,0,1,0,0,0,0,0
2,הצגה עצמיתתחילה תתבקש לפרט את המחלות או הליקוי...,0,1,19,1,9,4,2,0,0,...,0,0,0,0,1,0,0,0,0,0
3,מגמת משרד החינוך הינה לשלב את הילד בעל הצרכים ...,0,1,25,1,16,3,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"בייעוץ כזה יובאו בחשבון המסלול, משך הטיול, אופ...",0,1,24,1,18,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,"הורים יקרים,במדריך הזה מפורט התהליך העומד בפני...",0,1,22,1,14,3,4,0,1,...,0,0,0,0,0,0,0,0,0,0
517,מהיום ניתן לקבל את חשבון הארנונה ישירות לדואר...,0,1,11,1,6,2,3,0,0,...,0,0,0,0,0,0,0,0,0,0
518,".תורם לאיכות הסביבה, קבלת חשבון ארנונה תקופתי ...",0,1,18,1,12,3,2,0,0,...,0,0,0,0,0,0,0,0,0,0
519,ניתן לקבל את חשבון הארנונה למייל במילוי טופס.,0,1,7,1,5,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# NN_l = []
# VB_l = []
# JJ_l = []
# PREPOSITION_l = []
# DEF_l = []
# CONJ_l = []
# BN_l = []
# NNP_l = []
# NNT_l = []
# TEMP_l = []
# RB_l = []
# CC_l = []
# AT_l = []
# QW_l = []
# REL_l = []
# PRP_l = []
# IN_l = []
# DTT_l = []
# DT_l = []
# MD_l = []

# t_depth = []
# mdd = []

In [4]:
f57 = pd.read_csv('first_511.csv')
print('Shape:', f57.shape)
f57.head()

Shape: (511, 22)


,mdd,t_depth,NN,VB,JJ,PREPOSITION,DEF,CONJ,BN,NNP,...,RB,CC,AT,QW,REL,PRP,IN,DTT,DT,MD
0,3.75,4,4,0,0,2,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14.82,34,16,6,8,14,4,6,4,6,...,2,2,0,0,0,0,0,0,0,0
2,14.41,28,12,4,2,2,12,4,4,4,...,0,2,2,2,2,4,2,0,0,0
3,18.42,36,16,6,4,8,20,0,0,6,...,2,2,2,0,2,0,4,2,0,0
4,19.70,43,16,2,8,6,28,2,0,14,...,0,0,0,2,4,2,0,2,2,0


In [5]:
NN_l = f57['NN'].tolist()
VB_l = f57['VB'].tolist()
JJ_l = f57['JJ'].tolist()
PREPOSITION_l = f57['PREPOSITION'].tolist()
DEF_l = f57['DEF'].tolist()
CONJ_l = f57['CONJ'].tolist()
BN_l = f57['BN'].tolist()
NNP_l = f57['NNP'].tolist()
NNT_l = f57['NNT'].tolist()
TEMP_l = f57['TEMP'].tolist()
RB_l = f57['RB'].tolist()
CC_l = f57['CC'].tolist()
AT_l = f57['AT'].tolist()
QW_l = f57['QW'].tolist()
REL_l = f57['REL'].tolist()
PRP_l = f57['PRP'].tolist()
IN_l = f57['IN'].tolist()
DTT_l = f57['DTT'].tolist()
DT_l = f57['DT'].tolist()
MD_l = f57['MD'].tolist()

t_depth = f57['t_depth'].tolist()
mdd = f57['mdd'].tolist()

In [6]:
len(NN_l)

511

In [19]:
#s = bef.sentence[520]
s = bef.sentence[520]
print(s)  
no_q = re.sub("\"", "", s)
text = no_q

localhost_yap = "http://localhost:8000/yap/heb/joint"
data = '{{"text": "{}  "}}'.format(text).encode('utf-8')  # input string ends with two space characters
headers = {'content-type': 'application/json'}
response = requests.get(url=localhost_yap, data=data, headers=headers)
json_response = response.json()

heads_tree = []
heads = []
dep_tree = json_response['dep_tree']
dep_tree2 = dep_tree.split()
# print(dep_tree)
# print(dep_tree2[:10])
for i in range(len(dep_tree2)):
    if i<len(dep_tree2)-1:
        if dep_tree2[7]=='_' and dep_tree2[8]=='_':
            heads_tree.append(dep_tree2[:9])
            dep_tree2 = dep_tree2[9::]
        elif dep_tree2[8]=='_' and dep_tree2[9]=='_':
            heads_tree.append(dep_tree2[:10])
            dep_tree2 = dep_tree2[10::]
#             print(i)
#             print('len: ', len(dep_tree2))
#             print(dep_tree2)
        else:
            heads_tree.append(dep_tree2[:9])
            dep_tree2 = dep_tree2[9::]
#             print(str(i)+'  else')
#             print('len: ', len(dep_tree2))
#             print(dep_tree2)
                
for i in range(len(heads_tree)):
    if heads_tree[i][6].isnumeric():
        heads.append(int(heads_tree[i][6]))
    elif heads_tree[i][5].isnumeric():
        heads.append(int(heads_tree[i][5]))
        
t_depth.append(max(heads))
if (len(heads)-1)==0:
    mdd.append(float("{:.2f}".format(sum(heads))))
else:
    mdd.append(float("{:.2f}".format(sum(heads)/(len(heads)-1))))

NN = 0
VB = 0
JJ = 0
PREPOSITION = 0
DEF = 0
CONJ = 0
BN = 0
NNP = 0
NNT = 0
TEMP = 0
RB = 0
CC = 0
AT = 0
QW = 0
REL = 0
PRP = 0
IN = 0
DTT = 0
DT = 0
MD = 0
  
for w in range(len(dep_tree.split())):
    if dep_tree.split()[w] == "NN":
        NN=NN+1; 
    if dep_tree.split()[w] == "VB":
        VB=VB+1;
    if dep_tree.split()[w] == "JJ":
        JJ=JJ+1;
    if dep_tree.split()[w] == "PREPOSITION":
        PREPOSITION=PREPOSITION+1;
    if dep_tree.split()[w] == "DEF":
        DEF=DEF+1;
    if dep_tree.split()[w] == "CONJ":
        CONJ=CONJ+1;
    if dep_tree.split()[w] == "BN":
        BN=BN+1;
    if dep_tree.split()[w] == "NNP":
        NNP=NNP+1;
    if dep_tree.split()[w] == "NNT":
        NNT=NNT+1;
    if dep_tree.split()[w] == "TEMP":
        TEMP=TEMP+3;
    if dep_tree.split()[w] == "RB":
        RB=RB+1;
    if dep_tree.split()[w] == "CC":
        CC=CC+1;
    if dep_tree.split()[w] == "AT":
        AT=AT+1;
    if dep_tree.split()[w] == "QW":
        QW=QW+1;
    if dep_tree.split()[w] == "REL":
        REL=REL+1;
    if dep_tree.split()[w] == "PRP":
        PRP=PRP+1;
    if dep_tree.split()[w] == "IN":
        IN=IN+1;
    if dep_tree.split()[w] == "DTT":
        DTT=DTT+1;
    if dep_tree.split()[w] == "DT":
        DT=DT+1;
    if dep_tree.split()[w] == "MD":
        MD=MD+1;

    
NN_l.append(NN)
VB_l.append(VB)
JJ_l.append(JJ)
PREPOSITION_l.append(PREPOSITION)
DEF_l.append(DEF)
CONJ_l.append(CONJ)
BN_l.append(BN)
NNP_l.append(NNP)
NNT_l.append(NNT)
TEMP_l.append(TEMP)
RB_l.append(RB)
CC_l.append(CC)
AT_l.append(AT)
QW_l.append(QW)
REL_l.append(REL)
PRP_l.append(PRP)
IN_l.append(IN)
DTT_l.append(DTT)
DT_l.append(DT)
MD_l.append(MD)

In [20]:
print(len(t_depth))

521


In [31]:
#print(t_depth)

In [32]:
#print(mdd)

In [23]:
# for i in range(2):
#     t_depth.pop()
#     mdd.pop()

#     NN_l.pop()
#     VB_l.pop()
#     JJ_l.pop()
#     PREPOSITION_l.pop()
#     DEF_l.pop()
#     CONJ_l.pop()
#     BN_l.pop()
#     NNP_l.pop()
#     NNT_l.pop()
#     TEMP_l.pop()
#     RB_l.pop()
#     CC_l.pop()
#     AT_l.pop()
#     QW_l.pop()
#     REL_l.pop()
#     PRP_l.pop()
#     IN_l.pop()
#     DTT_l.pop()
#     DT_l.pop()
#     MD_l.pop()

In [24]:
#print(mdd[:377])

In [25]:
# i=376
# (t_depth[i], mdd[i])

In [26]:
# temp = t_depth[:172]
# print(temp)
# temp2 = t_depth[173:]
# print(temp2)
# temp3 = temp + temp2 + [61]
# print(temp3)
# print(len(temp3))

In [27]:
df = pd.DataFrame({'mdd' : mdd,
                    't_depth' : t_depth,
                    'NN' : NN_l,
                    'VB' : VB_l,
                    'JJ' : JJ_l,
                    'PREPOSITION' : PREPOSITION_l,
                    'DEF' : DEF_l,
                    'CONJ' : CONJ_l,
                    'BN' : BN_l,
                    'NNP' : NNP_l,
                    'NNT' : NNT_l,
                    'TEMP' : TEMP_l,
                    'RB' : RB_l,
                    'CC' : CC_l,
                    'AT' : AT_l,
                    'QW' : QW_l,
                    'REL' : REL_l,
                    'PRP' : PRP_l,
                    'IN' : IN_l,
                    'DTT' : DTT_l,
                    'DT' : DT_l,
                    'MD' : MD_l})

df.head()

,mdd,t_depth,NN,VB,JJ,PREPOSITION,DEF,CONJ,BN,NNP,...,RB,CC,AT,QW,REL,PRP,IN,DTT,DT,MD
0,3.75,4,4,0,0,2,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14.82,34,16,6,8,14,4,6,4,6,...,2,2,0,0,0,0,0,0,0,0
2,14.41,28,12,4,2,2,12,4,4,4,...,0,2,2,2,2,4,2,0,0,0
3,18.42,36,16,6,4,8,20,0,0,6,...,2,2,2,0,2,0,4,2,0,0
4,19.70,43,16,2,8,6,28,2,0,14,...,0,0,0,2,4,2,0,2,2,0


In [28]:
df.shape

(521, 22)

In [29]:
print('Time elapsed in (hh:mm:ss.ms): "{}"'.format(datetime.now() - start_time))

Time elapsed in (hh:mm:ss.ms): "0:03:20.861050"


In [30]:
df.to_csv('final_520.csv', index = False)